# Prepare ENV/Load libraries

In [1]:
# !rm -rf tmp && git clone https://github.com/chuan-khuna/my-python-utils.git tmp && cp -R tmp/utils ./ && cp -R tmp/fonts ./ && rm -rf tmp

In [2]:
!python3 --version

Python 3.8.16


In [3]:
!pip install matplotlib seaborn -Uq
!pip install pythainlp -q
!pip install keras-tuner -q
!pip install pyYAML -q

     |████████████████████████████████| 9.4 MB 7.8 MB/s 
     |████████████████████████████████| 288 kB 30.1 MB/s 
     |████████████████████████████████| 965 kB 49.3 MB/s 
     |████████████████████████████████| 295 kB 54.8 MB/s 
     |████████████████████████████████| 9.6 MB 7.2 MB/s 
     |████████████████████████████████| 135 kB 7.1 MB/s 
     |████████████████████████████████| 1.6 MB 47.6 MB/s 


## Mount Google Drive

In [4]:
import os
from google.colab import drive

drive.mount('/content/gdrive')
google_drive_path = "/content/gdrive/MyDrive/"

Mounted at /content/gdrive


In [5]:
# change directory to the project path

project_path = os.path.join(google_drive_path, "Colab Notebooks/SER ME/")
os.chdir(project_path)
os.listdir("./")

['utils',
 'fonts',
 'meta_data',
 'dataset',
 '.git',
 '.env',
 'config.yml',
 'libs',
 'README.md',
 'models',
 'Notebook Template.ipynb',
 '.gitignore',
 'logs',
 'HyperParams Tuning Train LSTM.ipynb',
 'Preprocess Json to CSV.ipynb',
 'Untitled0.ipynb']

# Import Libraries

In [6]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib
from matplotlib import patheffects
import seaborn as sns

import json
import yaml
import re
from tqdm.notebook import trange, tqdm

import datetime

In [7]:
import tensorflow as tf
import tensorflow_datasets as tfds
import keras_tuner

seed_ = 20200218
tf.random.set_seed(seed_)
np.random.seed(seed_)

from sklearn.metrics import confusion_matrix, classification_report

In [8]:
from utils.vis_utils import *

font_dir = [f"{google_drive_path}/code_assets/fonts/"]
mpl_import_fonts(font_dir)


Bad key "xtick.labelcolor" on line 70 in
/content/gdrive/MyDrive/Colab Notebooks/SER ME/utils/vis_utils/../vis_utils/theme.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key "ytick.labelcolor" on line 71 in
/content/gdrive/MyDrive/Colab Notebooks/SER ME/utils/vis_utils/../vis_utils/theme.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.2/matplotlibrc.template
or from the matplotlib source distribution


## Ensure that matplotlib can use subplot mosaic

In [9]:
def check_version(version_str, major, minor):
    print(version_str)
    version = [int(i) for i in version_str.split('.')]
    assert version[0] >= major and version[1] >= minor

check_version(matplotlib.__version__, 3, 6)
check_version(sns.__version__, 0, 12)

del check_version

matplotlib.__version__, sns.__version__, tf.__version__

3.2.2


AssertionError: ignored

## View hardware spec

In [ ]:
!nvidia-smi

In [ ]:
tf.config.list_physical_devices('GPU')

for device in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(device, True)

# Preprocess Json to CSV

In [ ]:
with open("config.yml") as f:
    config = yaml.load(f, yaml.Loader)

In [ ]:
raw_df = pd.read_json("./meta_data/emotion_label.json", orient='index')
raw_df = raw_df.reset_index().join(pd.json_normalize(raw_df[0]))
raw_df = raw_df.drop(columns=[0, 'annotated'])
raw_df = raw_df.rename(columns={'index': 'file'})

In [ ]:
raw_df = pd.read_csv("./meta_data/emotion_label.csv")

In [ ]:
raw_df

# Search for available files

In [ ]:
audio_files = []

for path, subdirs, files in os.walk("./dataset/"):
    for name in files:
        file = os.path.join(path, name)
        audio_files.append(file)

audio_files = [file for file in audio_files if file.endswith(".flac")]

# Merge available files with corresponding labels

In [ ]:
df = pd.DataFrame({'path': audio_files})

In [ ]:
df['file'] = df['path'].apply(lambda x: x.split('/')[-1])

In [ ]:
df = df.join(raw_df.set_index('file'), on='file')

In [ ]:
df

In [ ]:
df.to_csv("./meta_data/dataset.csv", index=False)